## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-06-13-00-00 +0000,missionlocal,Elvis,https://missionlocal.org/2025/10/sf-photos-elvis/
1,2025-10-06-12-59-00 +0000,wsj,Verizon Names Lead Director Daniel Schulman as...,https://www.wsj.com/business/telecom/verizon-n...
2,2025-10-06-12-58-09 +0000,bbc,Eswatini accepts 10 US deportees despite legal...,https://www.bbc.com/news/articles/cn5qdx260lzo...
3,2025-10-06-12-54-53 +0000,nyt,"Paramount Buys Bari Weiss’s Free Press, Starti...",https://www.nytimes.com/2025/10/06/business/me...
4,2025-10-06-12-52-10 +0000,wapo,"Gaza ceasefire talks to begin in Egypt, as Isr...",https://www.washingtonpost.com/world/2025/10/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2384/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
76,trump,38
21,new,14
25,gaza,13
513,shutdown,10
98,government,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
220,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...,149
190,2025-10-05-18-37-30 +0000,latimes,Judge blocks Trump from deploying California N...,https://www.latimes.com/world-nation/story/202...,98
28,2025-10-06-11-20-04 +0000,startribune,Federal judge temporarily blocks Trump adminis...,https://www.startribune.com/california-governo...,87
200,2025-10-05-17-22-31 +0000,nypost,Kristi Noem slams ‘delusional’ Chicago mayor w...,https://nypost.com/2025/10/05/us-news/kristi-n...,85
94,2025-10-06-07-17-48 +0000,nypost,Federal judge blocks Trump’s National Guard de...,https://nypost.com/2025/10/06/us-news/federal-...,85


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
220,149,2025-10-05-14-41-00 +0000,wsj,President Trump pressed ahead with plans to mo...,https://www.wsj.com/politics/policy/trump-auth...
24,49,2025-10-06-11-59-27 +0000,wapo,Prime minister of France resigns hours after n...,https://www.washingtonpost.com/world/2025/10/0...
143,40,2025-10-05-23-07-07 +0000,bbc,Gisèle Pelicot to face one of her rapists in c...,https://www.bbc.com/news/articles/cdx2d9lynk1o...
136,39,2025-10-05-23-53-33 +0000,bbc,Trump urges mediators to 'move fast' as key Ga...,https://www.bbc.com/news/articles/cj3y6g43248o...
98,36,2025-10-06-06-52-43 +0000,nypost,California biotech tycoon found guilty in murd...,https://nypost.com/2025/10/06/us-news/californ...
5,33,2025-10-06-12-47-00 +0000,wsj,Treasury Secretary Scott Bessent has appointed...,https://www.wsj.com/politics/policy/bessent-pi...
200,29,2025-10-05-17-22-31 +0000,nypost,Kristi Noem slams ‘delusional’ Chicago mayor w...,https://nypost.com/2025/10/05/us-news/kristi-n...
219,28,2025-10-05-14-45-55 +0000,nypost,Virginia Beach Dems back Virginia AG nominee J...,https://nypost.com/2025/10/05/us-news/dems-bac...
31,26,2025-10-06-11-05-26 +0000,nypost,"Three scientists — 2 Americans, 1 Japanese — t...",https://nypost.com/2025/10/06/world-news/three...
46,26,2025-10-06-10-23-28 +0000,bbc,'I was lucky to get out': Everest hikers battl...,https://www.bbc.com/news/articles/cj4ykkgxqwko...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
